# >프로그램< 기본 라이브러리 로드

In [25]:
from urllib.request import urlretrieve
urlretrieve("https://raw.githubusercontent.com/c0z0c/jupyter_hangul/refs/heads/beta/helper_utils.py", "helper_utils.py")
import importlib
import helper_utils as hu
importlib.reload(hu)
from helper_utils import *

from urllib.request import urlretrieve; urlretrieve("https://raw.githubusercontent.com/c0z0c/jupyter_hangul/refs/heads/beta/helper_c0z0c_dev.py", "helper_c0z0c_dev.py")
import importlib
import helper_c0z0c_dev as helper
importlib.reload(helper)

# 표준/유틸
from pathlib import Path
import random
import math
import os
import sys
import json
import yaml
import zipfile
import shutil
import logging
import re
import unicodedata
import html
import pickle
from collections import OrderedDict, Counter, defaultdict
from typing import Dict, List, Tuple, Optional
from abc import ABC, abstractmethod

# 데이터/시각화
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import xml.etree.ElementTree as ET
from datetime import datetime, timezone, timedelta
import pytz

# Scikit-learn (필요 모듈만 그룹화)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.datasets import (
    fetch_california_housing, load_iris, make_moons, make_circles,
    load_breast_cancer, load_wine
)
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, average_precision_score

# 이미지 관련
from PIL import Image, ImageFilter, ImageDraw
import albumentations as A
import IPython.display as display
from tqdm.notebook import tqdm

# PyTorch 관련
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset, DataLoader, TensorDataset, RandomSampler
from torchvision.transforms import v2
from torchvision.datasets import CocoDetection
from torchvision.transforms import functional as TF
from torch.nn import CrossEntropyLoss

# --- 전역 설정 (규약 준수: 변수명 유지) ---
__kst = pytz.timezone('Asia/Seoul')
__device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
__device_cpu = torch.device('cpu')

# 재현성 및 시드 고정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if __device.type == 'cuda':
    torch.cuda.manual_seed_all(SEED)
    # 결정론적 설정 (필요 시 해제 가능)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.info(f"라이브러리 로드 완료. 사용장치: {__device}")

helper_utils.py loaded


2025-10-13 17:32:34 [I] 라이브러리 로드 완료. 사용장치: cpu


🌐 https://c0z0c.github.io/jupyter_hangul
✅ 설정 완료: 한글 폰트, plt 전역 등록, pandas 확장, 캐시 기능
pd commit 저장 경로 = d:\GoogleDrive\homepage\스프린트미션\스터디\논문검색_유사도


In [46]:
class AIHubShell:
    def __init__(self, DEBUG=False, download_dir=None, version="0.6"):
        self.VERSION = version
        self.BASE_URL = "https://api.aihub.or.kr"
        self.LOGIN_URL = f"{self.BASE_URL}/api/keyValidate.do"
        self.BASE_DOWNLOAD_URL = f"{self.BASE_URL}/down/{self.VERSION}"
        self.MANUAL_URL = f"{self.BASE_URL}/info/api.do"
        self.BASE_FILETREE_URL = f"{self.BASE_URL}/info"
        self.DATASET_URL = f"{self.BASE_URL}/info/dataset.do"
        self.DATAPCKAGE_URL = f"{self.BASE_URL}/info/datapckage.do"
        self.BASE_PCKAGETREE_URL = f"{self.BASE_URL}/info/pckage"
        self.BASE_PCKAGE_DOWNLOAD_URL = f"{self.BASE_URL}/down/pckage/{self.VERSION}"
        self.DEBUG = DEBUG
        self.download_dir = download_dir if download_dir else "."
                
    def help(self):
        """AIHubShell 클래스 사용법 출력"""
        print("=" * 80)
        print("                        AIHubShell 클래스 사용 가이드")
        print("=" * 80)
        print()
        
        print("초기화")
        print("  AIHubShell(DEBUG=False, download_dir=None)")
        print("    DEBUG: True로 설정하면 상세 로그 출력")
        print("    download_dir: 다운로드 경로 지정 (기본값: 현재 경로)")
        print()
        
        print("데이터셋 조회")
        print("  .dataset_info()                    # 전체 데이터셋 목록 조회")
        print("  .dataset_search('검색어')          # 특정 이름 포함 데이터셋 검색")
        print("  .dataset_search('검색어', tree=True) # 검색 + 파일 트리 조회")
        print("  .list_info(datasetkey=576)         # 특정 데이터셋의 파일 목록")
        print("  .json_info(datasetkey=576)         # JSON 형태로 파일 구조 반환")
        print()
        
        print("데이터 패키지 조회")
        print("  .datapckage_info()                 # 전체 데이터 패키지 목록 조회")
        print("  .datapckage_info(datapckagekey=123) # 특정 패키지 파일 트리 조회")
        print()
        
        print("다운로드")
        print("  .download_dataset(apikey, datasetkey, filekeys='all')")
        print("    apikey: AI Hub API 키")
        print("    datasetkey: 데이터셋 번호")
        print("    filekeys: 파일키 ('all' 또는 '66065,66083' 형태)")
        print("    overwrite: 기존 파일 덮어쓰기 여부 (기본값: False)")
        print("    resume: 중단된 다운로드 이어받기 (기본값: True)")
        print()
        print("  .download_datapckage(apikey, datapckagekey, filekeys='all')")
        print("    데이터 패키지 다운로드 (옵션은 download_dataset과 동일)")
        print()
        
        print("기타 기능")
        print("  .print_usage()                     # AI Hub API 상세 사용법")
        print("  .help()                            # 이 도움말")
        print()
        
        print("사용 예시")
        print("  # 1. 인스턴스 생성")
        print("  aihub = AIHubShell(DEBUG=True, download_dir='./data')")
        print()
        print("  # 2. 경구약제 데이터셋 검색")
        print("  aihub.dataset_search('경구약제')")
        print()
        print("  # 3. 데이터셋 576의 파일 목록 확인")
        print("  aihub.list_info(datasetkey=576)")
        print()
        print("  # 4. 특정 파일들만 다운로드")
        print("  aihub.download_dataset(")
        print("      apikey='YOUR_API_KEY',")
        print("      datasetkey=576,")
        print("      filekeys='66065,66083'")
        print("  )")
        print()
        print("  # 5. 전체 데이터셋 다운로드 (이어받기 지원)")
        print("  aihub.download_dataset(")
        print("      apikey='YOUR_API_KEY',")
        print("      datasetkey=576,")
        print("      filekeys='all',")
        print("      resume=True")
        print("  )")
        print()
        print("  # 6. 데이터 패키지 목록 조회 및 다운로드")
        print("  aihub.datapckage_info()")
        print("  aihub.download_datapckage(")
        print("      apikey='YOUR_API_KEY',")
        print("      datapckagekey=123")
        print("  )")
        print()
        
        print("주의사항")
        print("  - API 키는 AI Hub에서 발급받아야 합니다")
        print("  - 대용량 파일 다운로드 시 충분한 저장 공간을 확보하세요")
        print("  - overwrite=False일 때 기존 파일은 자동으로 건너뜁니다")
        print("  - resume=True일 때 중단된 다운로드를 자동으로 이어받습니다")
        print("  - 네트워크 상태에 따라 다운로드 시간이 달라질 수 있습니다")
        print()
        
        print("추가 정보")
        print("  AI Hub API 공식 문서: https://aihub.or.kr")
        print("  문제 발생 시 DEBUG=True로 설정하여 상세 로그를 확인하세요")
        print("=" * 80)
                        
    def print_usage(self):
        """사용법 출력"""
        try:
            response = requests.get(self.MANUAL_URL)
            manual = response.text
            
            if self.DEBUG:
                print("API 원본 응답:")
                print(manual)            
            
            # JSON 파싱하여 데이터 추출
            try:
                manual = re.sub(r'("FRST_RGST_PNTTM":)([0-9\- :\.]+)', r'\1"\2"', manual)
                manual_data = json.loads(manual)
                if self.DEBUG:
                    print("JSON 파싱 성공")
                    
                if 'result' in manual_data and len(manual_data['result']) > 0:
                    print(manual_data['result'][0].get('SJ', ''))
                    print()
                    print("ENGL_CMGG\t KOREAN_CMGG\t\t\t DETAIL_CN")
                    print("-" * 80)
                    
                    for item in manual_data['result']:
                        engl = item.get('ENGL_CMGG', '')
                        korean = item.get('KOREAN_CMGG', '')
                        detail = item.get('DETAIL_CN', '').replace('\\n', '\n').replace('\\t', '\t')
                        print(f"{engl:<10}\t {korean:<15}\t|\t {detail}\n")
            except json.JSONDecodeError as e:
                if self.DEBUG:
                    print("JSON 파싱 오류:", e)
                else:
                    print("API 응답 파싱 오류")
        except requests.RequestException as e:
            print(f"API 요청 오류: {e}")
    
    def _merge_parts(self, target_dir):
        """part 파일들을 병합"""
        target_path = Path(target_dir)
        part_files = list(target_path.glob("*.part*"))
        
        if not part_files:
            return
            
        # prefix별로 그룹화
        prefixes = {}
        for part_file in part_files:
            match = re.match(r'(.+)\.part(\d+)$', part_file.name)
            if match:
                prefix = match.group(1)
                part_num = int(match.group(2))
                if prefix not in prefixes:
                    prefixes[prefix] = []
                prefixes[prefix].append((part_num, part_file))
        
        # 각 prefix별로 병합
        for prefix, parts in prefixes.items():
            print(f"Merging {prefix} in {target_dir}")
            parts.sort(key=lambda x: x[0])  # part 번호로 정렬
            
            output_path = target_path / prefix
            with open(output_path, 'wb') as output_file:
                for _, part_file in parts:
                    with open(part_file, 'rb') as input_file:
                        shutil.copyfileobj(input_file, output_file)
            
            # part 파일들 삭제
            for _, part_file in parts:
                part_file.unlink()
                
    def _merge_parts_all(self, base_path="."):
        """모든 하위 폴더의 part 파일들을 병합"""
        if self.DEBUG:
            print("병합 중입니다...")
        for root, dirs, files in os.walk(base_path):
            part_files = [f for f in files if '.part' in f]
            if part_files:
                self._merge_parts(root)
        if self.DEBUG:
            print("병합이 완료되었습니다.")
    
    def download_dataset(self, apikey, datasetkey, filekeys="all", overwrite=False, resume=True):
        """데이터셋 다운로드 (옵션: 덮어쓰기, 이어받기)"""
        def _parse_size(size_str):
            """'92 GB', '8 MB' 등 문자열을 바이트 단위로 변환"""
            size_str = size_str.strip().upper()
            if 'GB' in size_str:
                return float(size_str.replace('GB', '').strip()) * 1024**3
            elif 'MB' in size_str:
                return float(size_str.replace('MB', '').strip()) * 1024**2
            elif 'KB' in size_str:
                return float(size_str.replace('KB', '').strip()) * 1024
            elif 'B' in size_str:
                return float(size_str.replace('B', '').strip())
            return 0
        
        download_path = Path(self.download_dir)
        download_tar_path = download_path / "download.tar"
        
        # 기존 download.tar 백업 (이어받기가 아닌 경우)
        if download_tar_path.exists() and not resume:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            backup_path = download_path / f"download_{timestamp}.tar"
            shutil.move(str(download_tar_path), str(backup_path))
            print(f"기존 파일을 {backup_path.name}로 백업했습니다.")
        
        download_list = self.list_info(datasetkey=datasetkey, filekeys=filekeys, print_out=False)
        
        # 이미 존재하는 파일은 제외
        keys_to_download = []
        for key, info in download_list.items():
            extracted_file_path = os.path.join(self.download_dir, info.path)
            if not overwrite and os.path.exists(extracted_file_path):
                print(f"파일 발견: {extracted_file_path}")
                if self.DEBUG:
                    print("다운로드를 생략합니다.")
                continue
            
            # 압축 해지 하고 용량 이슈로 인하여 zip파일은 삭제 되었다.
            if not overwrite and os.path.exists(extracted_file_path + ".unzip"):
                print(f"파일 발견 unzip: {extracted_file_path}.unzip")
                if self.DEBUG:
                    print("다운로드를 생략합니다.")
                continue
            
            keys_to_download.append(str(key))

        # 다운로드할 filekeys가 없으면 종료
        if not keys_to_download:
            print("모든 파일이 이미 존재합니다.")
            extracted_files = []
            for key, info in download_list.items():
                file_path = os.path.join(self.download_dir, info.path)
                if os.path.exists(file_path):
                    extracted_files.append(file_path)
            print("다운로드 파일 목록:", extracted_files)
            return extracted_files            

        # 헤더와 파라미터 기본 설정
        headers = {"apikey": apikey}
        params = {"fileSn": ",".join(keys_to_download)}
        
        # 이어받기 설정
        mode = "wb"
        existing_size = 0
        if resume and download_tar_path.exists():
            existing_size = download_tar_path.stat().st_size
            mode = "ab"
            headers["Range"] = f"bytes={existing_size}-"
            if self.DEBUG:
                print(f"기존 파일 발견. {existing_size} 바이트부터 이어받기...")
        
        response_head = requests.head(f"{self.BASE_DOWNLOAD_URL}/{datasetkey}.do", headers=headers, params=params)
        if "content-length" in response_head.headers:
            total_size = int(response_head.headers.get('content-length', 0))
        else:
            total_size = 0
            if self.DEBUG:
                print("content-length 헤더가 없습니다. 전체 크기 알 수 없음.")
                print("HEAD 응답 헤더:", response_head.headers)

        if total_size == 0:
            total_size = int(sum(_parse_size(info.size) for info in download_list.values()))
            if self.DEBUG:
                print(f"download_list 기반 추정 total_size: {total_size / (1024**3):.2f} GB")
                
        # 실제 다운로드
        if self.DEBUG:
            print("다운로드 시작...")
            
        os.makedirs(download_path, exist_ok=True)
        response = requests.get(
            f"{self.BASE_DOWNLOAD_URL}/{datasetkey}.do",
            headers=headers,
            params=params,
            stream=True
        )

        if response.status_code in [200, 206]:
            
            with open(download_tar_path, mode) as f, tqdm(
                total=total_size, 
                unit='B', 
                unit_scale=True, 
                desc="Downloading", 
                mininterval=3.0,  # 3초마다 갱신
                initial=(existing_size if mode == "ab" else 0)
            ) as pbar:
                update_count = 1000
                downloaded = existing_size if mode == "ab" else 0
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
                    downloaded += len(chunk)
                    pbar.update(len(chunk))
                    if update_count <= 0:
                        pbar.set_postfix_str(f"{downloaded / (1024**2):.2f}MB / {total_size / (1024**2):.2f}MB")
                        update_count = 1000
                    update_count -= 1
                f.flush()
            
            if self.DEBUG:
                print("압축 해제 중...")
            with tarfile.open(download_tar_path, "r") as tar:
                tar.extractall(path=download_path)
            self._merge_parts_all(str(download_path))
            download_tar_path.unlink()
            
            print("다운로드 완료!")
        else:
            print(f"Download failed with HTTP status {response.status_code}.")
            print("Error msg:")
            print(response.text)
            if download_tar_path.exists():
                download_tar_path.unlink()
                
        extracted_files = []
        for key, info in download_list.items():
            file_path = os.path.join(self.download_dir, info.path)
            if os.path.exists(file_path):
                extracted_files.append(file_path)
        print("다운로드 파일 목록:", extracted_files)
        return extracted_files            
                
    def list_info(self, datasetkey=None, filekeys="all", print_out=True):
        """데이터셋 파일 정보 조회 (filekeys, 파일명, 사이즈 출력 및 딕셔너리 반환)"""
        resjson = self.json_info(datasetkey=datasetkey)
        
        # 파일 정보를 담을 딕셔너리
        file_info_dict = {}
        
        def extract_files(structure):
            """재귀적으로 파일 정보 추출"""
            for item in structure:
                if item["type"] == "file" and "filekey" in item:
                    filekey = int(item["filekey"])
                    file_info_dict[filekey] = {
                        "filekey": item["filekey"],
                        "filename": item["name"],
                        "size": item["size"],
                        "path": item["path"],
                        "deep": item["deep"]
                    }
                elif item["type"] == "directory" and "children" in item:
                    extract_files(item["children"])
        
        # JSON 구조에서 파일 정보 추출
        extract_files(resjson["structure"])
        
        # filekeys 처리
        if filekeys == "all":
            filtered_files = file_info_dict
        else:
            # 쉼표로 구분된 filekeys 파싱
            requested_keys = []
            for key in filekeys.split(','):
                try:
                    requested_keys.append(int(key.strip()))
                except ValueError:
                    continue
            
            # 요청된 filekey만 필터링
            filtered_files = {k: v for k, v in file_info_dict.items() if k in requested_keys}
        
        # 출력
        if print_out:
            print(f"Dataset: {datasetkey}")
            print("=" * 80)
            print(f"{'FileKey':<8} {'Filename':<30} {'Size':<10} {'Path'}")
            print("-" * 80)
            
            for filekey, info in sorted(filtered_files.items()):
                print(f"{info['filekey']:<8} {info['filename']:<30} {info['size']:<10} {info['path']}")
            
            print(f"\n총 {len(filtered_files)}개 파일")
        
        # 딕셔너리 반환 (FileInfo 객체 형태로)
        class FileInfo:
            def __init__(self, filekey, filename, size, path, deep):
                self.filekey = filekey
                self.filename = filename
                self.size = size
                self.path = path
                self.deep = deep
            
            def __str__(self):
                return f"FileInfo(filekey={self.filekey}, filename='{self.filename}', size='{self.size}' , path='{self.path}', deep={self.deep})"
            
            def __repr__(self):
                return self.__str__()
        
        result_dict = {}
        for filekey, info in filtered_files.items():
            result_dict[filekey] = FileInfo(
                filekey=info["filekey"],
                filename=info["filename"],
                size=info["size"],
                path=info["path"],
                deep=info["deep"]
            )
        
        return result_dict
        
    def dataset_info(self, datasetkey=None, datasetname=None):
        """데이터셋 목록 또는 파일 트리 조회"""
        if datasetkey:
            filetree_url = f"{self.BASE_FILETREE_URL}/{datasetkey}.do"
            print("Fetching file tree structure...")
            try:
                response = requests.get(filetree_url)
                # 인코딩 자동 감지
                response.encoding = response.apparent_encoding
                print(response.text)
            except requests.RequestException as e:
                print(f"API 요청 오류: {e}")
        else:
            print("Fetching dataset information...")
            try:
                response = requests.get(self.DATASET_URL)
                response.encoding = 'utf-8'
                print(response.text)
            except requests.RequestException as e:
                print(f"API 요청 오류: {e}")

    def dataset_search(self, datasetname=None, tree=False):
        """
        데이터셋 목록 또는 특정 이름이 포함된 데이터셋의 파일 트리 조회
        datasetname: 검색할 데이터셋 이름 (부분 일치)
        tree: True이면 해당 데이터셋의 파일 트리도 조회        
        """
        print("Fetching dataset information...")
        try:
            response = requests.get(self.DATASET_URL)
            response.encoding = 'utf-8'
            text = response.text
            if datasetname:
                # datasetname이 포함된 부분만 출력
                lines = text.splitlines()
                for line in lines:
                    if datasetname in line:
                        # 576, 경구약제 이미지 데이터
                        num, name = line.split(',', 1)
                        # 해당 데이터셋의 파일 트리 조회
                        if tree:
                            self.dataset_info(datasetkey=num.strip())
                        else:
                            print(line)
            else:
                print(text)
        except requests.RequestException as e:
            print(f"API 요청 오류: {e}")

    def datapckage_info(self, datapckagekey=None):
        """데이터 패키지 목록 또는 파일 트리 조회"""
        if datapckagekey:
            pckagetree_url = f"{self.BASE_PCKAGETREE_URL}/{datapckagekey}.do"
            print("Fetching datapckage file tree structure...")
            try:
                response = requests.get(pckagetree_url)
                response.encoding = response.apparent_encoding
                print(response.text)
            except requests.RequestException as e:
                print(f"API 요청 오류: {e}")
        else:
            print("Fetching datapckage information...")
            try:
                response = requests.get(self.DATAPCKAGE_URL)
                response.encoding = 'utf-8'
                print(response.text)
            except requests.RequestException as e:
                print(f"API 요청 오류: {e}")
    
    def download_datapckage(self, apikey, datapckagekey, filekeys="all", overwrite=False, resume=True):
        """데이터 패키지 다운로드 (옵션: 덮어쓰기, 이어받기)"""
        def _parse_size(size_str):
            """'92 GB', '8 MB' 등 문자열을 바이트 단위로 변환"""
            size_str = size_str.strip().upper()
            if 'GB' in size_str:
                return float(size_str.replace('GB', '').strip()) * 1024**3
            elif 'MB' in size_str:
                return float(size_str.replace('MB', '').strip()) * 1024**2
            elif 'KB' in size_str:
                return float(size_str.replace('KB', '').strip()) * 1024
            elif 'B' in size_str:
                return float(size_str.replace('B', '').strip())
            return 0
        
        download_path = Path(self.download_dir)
        download_tar_path = download_path / "download.tar"
        
        # 기존 download.tar 백업 (이어받기가 아닌 경우)
        if download_tar_path.exists() and not resume:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            backup_path = download_path / f"download_{timestamp}.tar"
            shutil.move(str(download_tar_path), str(backup_path))
            print(f"기존 파일을 {backup_path.name}로 백업했습니다.")
        
        # 헤더와 파라미터 설정
        headers = {"apikey": apikey}
        params = {"fileSn": filekeys}
        
        # 이어받기 설정
        mode = "wb"
        existing_size = 0
        if resume and download_tar_path.exists():
            existing_size = download_tar_path.stat().st_size
            mode = "ab"
            headers["Range"] = f"bytes={existing_size}-"
            if self.DEBUG:
                print(f"기존 파일 발견. {existing_size} 바이트부터 이어받기...")
        
        response_head = requests.head(f"{self.BASE_PCKAGE_DOWNLOAD_URL}/{datapckagekey}.do", headers=headers, params=params)
        if "content-length" in response_head.headers:
            total_size = int(response_head.headers.get('content-length', 0))
        else:
            total_size = 0
            if self.DEBUG:
                print("content-length 헤더가 없습니다. 전체 크기 알 수 없음.")
        
        # 실제 다운로드
        if self.DEBUG:
            print("다운로드 시작...")
        
        os.makedirs(download_path, exist_ok=True)
        response = requests.get(
            f"{self.BASE_PCKAGE_DOWNLOAD_URL}/{datapckagekey}.do",
            headers=headers,
            params=params,
            stream=True
        )
        
        if response.status_code in [200, 206]:
            with open(download_tar_path, mode) as f, tqdm(
                total=total_size,
                unit='B',
                unit_scale=True,
                desc="Downloading",
                mininterval=3.0,
                initial=(existing_size if mode == "ab" else 0)
            ) as pbar:
                downloaded = existing_size if mode == "ab" else 0
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
                    downloaded += len(chunk)
                    pbar.update(len(chunk))
                f.flush()
            
            if self.DEBUG:
                print("압축 해제 중...")
            with tarfile.open(download_tar_path, "r") as tar:
                tar.extractall(path=download_path)
            self._merge_parts_all(str(download_path))
            download_tar_path.unlink()
            
            print("다운로드 완료!")
        else:
            print(f"Download failed with HTTP status {response.status_code}.")
            print("Error msg:")
            print(response.text)
            if download_tar_path.exists():
                download_tar_path.unlink()

    def _get_depth_from_star_count(self, star_count, depth_mapping):
        """star_count 값을 깊이(deep)로 변환"""
        if star_count not in depth_mapping:
            # 새로운 star_count 값이면 배열에 추가
            depth_mapping.append(star_count)
            # 오름차순 정렬
            depth_mapping.sort()
        
        # 배열에서의 인덱스가 깊이
        return depth_mapping.index(star_count)

    def _json_line(self, line, json_obj, depth_mapping, path_stack, weight=0, deep=0):
        """파일 트리의 한 줄을 JSON 구조에 맞게 파싱하여 추가"""
        # 트리 구조 기호를 모두 *로 변경
        line = line.replace("├─", "└─")
        line = line.replace("│ ", "└─")
        while "    └─" in line:
            line = line.replace("    └─", "└─└─")
        while " └─" in line:
            line = line.replace(" └─", "└─")
        
        while "└─" in line:
            line = line.replace("└─", "*")
        
        # 앞부분의 * 개수와 문자열 추출
        star_count = 0
        for char in line:
            if char == '*':
                star_count += 1
            else:
                break
        clean_str = line.replace('*', '').strip()
        
        # star_count를 deep로 동적 변환
        deep = self._get_depth_from_star_count(star_count, depth_mapping)
        
        has_pipe = "|" in line
        
        # 파일/폴더 정보 추출
        if has_pipe:
            parts = clean_str.split('|')
            if len(parts) >= 3:
                filename = parts[0].strip()
                size = parts[1].strip()
                filekey = parts[2].strip()
                item_type = "file"
            else:
                filename = clean_str
                size = ""
                filekey = ""
                item_type = "directory"
        else:
            filename = clean_str
            size = ""
            filekey = ""
            item_type = "directory"
        
        # path_stack 조정 (현재 깊이에 맞게)
        while len(path_stack) > deep:
            path_stack.pop()
        
        # 현재 아이템 정보
        current_item = {
            "name": filename,
            "type": item_type,
            "deep": deep,
            "weight": star_count,
            "path": str(Path(*path_stack, filename)).replace(' ', '_')  # 공백을 언더스코어로 변경
        }
        
        if item_type == "file":
            current_item["size"] = size
            current_item["filekey"] = filekey
        else:
            current_item["children"] = []
        
        # JSON 구조에 추가 (배열 구조)
        current_array = json_obj
        for path_name in path_stack:
            # 해당 이름의 디렉토리를 찾아서 그 children 배열로 이동
            found = None
            for item in current_array:
                if item["name"] == path_name and item["type"] == "directory":
                    found = item
                    break
            if found:
                current_array = found["children"]
        
        # 현재 배열에 아이템 추가
        current_array.append(current_item)
        
        # 디렉토리인 경우 path_stack에 추가
        if item_type == "directory":
            path_stack.append(filename)
        
        return current_item

    def json_info(self, datasetkey=None):
        """데이터셋 목록 또는 파일 트리를 JSON 형태로 반환"""
        filetree_url = f"{self.BASE_FILETREE_URL}/{datasetkey}.do"        
        response = requests.get(filetree_url)
        response.encoding = response.apparent_encoding
        text = response.text
        
        # JSON 구조를 위한 딕셔너리
        result = {
            "datasetkey": datasetkey,
            "structure": []  # 배열로 변경
        }
        
        lines = text.splitlines()
        
        is_notify = True
        json_obj = []  # 루트 배열
        depth_mapping = []  # 각 파싱 세션마다 새로운 depth_mapping
        path_stack = []     # 현재 경로를 추적하는 스택

        for line in lines:
            if not line.strip() or '공지사항' in line or '=' in line:
                is_notify = False
                continue
            if is_notify:
                continue

            self._json_line(line, json_obj, depth_mapping, path_stack, weight=0, deep=0)
        
        result["structure"] = json_obj
        
        return result

def unzip(zipfile_list, remove_zip=False):
    import zipfile
    unzip_paths = []
    for zip_path in zipfile_list:
        if os.path.exists(zip_path) and os.path.isfile(zip_path):
            extract_dir = zip_path + ".unzip"
            unzip_paths.append(extract_dir)
            if not os.path.exists(extract_dir):
                os.makedirs(extract_dir, exist_ok=True)
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    members = zip_ref.namelist()
                    for member in tqdm(members, desc=f"압축 해제 중: {os.path.basename(zip_path)}", unit="file"):
                        zip_ref.extract(member, extract_dir)
                print(f"압축 해제 완료: {extract_dir}")
            else:
                print(f"이미 압축 해제됨: {extract_dir}")
            try:
                if remove_zip:
                    os.remove(zip_path)
            except FileNotFoundError as e:
                print(f"파일이 없음 {e} : {zip_path}")
        else:
            print(f"존재하지 않은 파일: {zip_path}")
    return unzip_paths

from typing import List, Optional
def zip_progress(files: List[str], zip_path: str, arc_names: Optional[List[str]] = None, compression = None) -> Optional[str]:
    """파일 목록을 ZIP으로 압축하면서 tqdm 프로그레스바 표시."""
    from pathlib import Path
    import zipfile
    try:
        pbar_kwargs = get_tqdm_kwargs() or {}
    except Exception:
        pbar_kwargs = {}

    if compression is None:
        compression = zipfile.ZIP_STORED

    paths = [Path(p) for p in files]
    exist_paths = [p for p in paths if p.exists()]
    if not exist_paths:
        print("zip_progress: 압축할 유효한 파일이 없습니다.")
        return None

    try:
        with zipfile.ZipFile(zip_path, 'w', compression) as zf:
            with tqdm(total=len(exist_paths), desc="Zipping", **pbar_kwargs) as pbar:
                for i, p in enumerate(exist_paths):
                    arc = None
                    if arc_names and i < len(arc_names):
                        arc = arc_names[i]
                    else:
                        arc = p.name
                    zf.write(str(p), arc)
                    pbar.update(1)
        return zip_path
    except Exception as e:
        print(f"zip_progress 실패: {e}")
        return None


In [27]:
if IS_COLAB:
    !conda install -c conda-forge faiss-gpu
else:
    !pip install -q faiss-cpu
!pip install -q sentence-transformers
logger.info("FAISS 및 Sentence-Transformers 설치 완료.")


2025-10-13 17:32:40 [I] FAISS 및 Sentence-Transformers 설치 완료.


In [28]:
#aihub = AIHubShell(download_dir=r"D:\dataset")
#aihub.help()

In [29]:
#aihub.dataset_search(datasetname="논문자료")
#aihub.dataset_search(datasetname="논문자료", tree=True)

In [30]:
# result = aihub.download_dataset(
#     apikey='10C8D561-AF9A-4BA6-86F7-8C0A713B210A',
#     datasetkey='90',
#     filekeys='all',
# )
# for r in result:
#      print(r)
#      unzip_path= unzip([r], remove_zip=False)
#      print_dir_tree(unzip_path[0])


In [31]:
# 1. 환경 설정 및 패키지 설치

from sentence_transformers import SentenceTransformer, util
import faiss

# GPU 확인
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")


CUDA Available: False
Device: CPU


In [32]:
# 2. AI Hub 데이터 로드 함수 (실제 데이터 구조 반영)
def load_aihub_papers(base_path=r"D:\dataset\018.논문자료_요약_데이터\01.데이터"):
    """
    AI Hub 한국어 논문 데이터 로드
    
    Args:
        base_path: 데이터셋 기본 경로
    
    Returns:
        papers: 논문 정보 리스트
    """
    papers = []
    
    # Training 데이터 경로
    train_paths = [
        os.path.join(base_path, "1._Training", "1._라벨링데이터_231101_add", "training_논문.zip.unzip"),
    ]
    
    # Validation 데이터 경로
    valid_paths = [
        os.path.join(base_path, "2._Validation", "1._라벨링데이터_231101_add", "validation_논문.zip.unzip"),
    ]
    
    # 모든 경로 합치기
    all_paths = train_paths + valid_paths
    
    print(f"데이터 로드 중...")
    total_loaded = 0
    
    for data_path in all_paths:
        if not os.path.exists(data_path):
            print(f"경로가 존재하지 않습니다: {data_path}")
            continue
        
        # JSON 파일 찾기
        json_files = []
        for root, dirs, files in os.walk(data_path):
            for file in files:
                if file.endswith('.json'):
                    json_files.append(os.path.join(root, file))
        
        print(f"처리 중: {data_path}")
        print(f"  찾은 JSON 파일: {len(json_files)}개")
        
        # 각 JSON 파일 파싱
        for json_file in tqdm(json_files, desc=f"로딩 중"):
            try:
                with open(json_file, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    
                    # 데이터 구조 확인 및 파싱
                    # JSON 구조: [ { "totalcount": ..., "data": [...] } ]
                    if isinstance(data, list) and len(data) > 0:
                        # 리스트의 첫 번째 요소가 딕셔너리인지 확인
                        if isinstance(data[0], dict) and 'data' in data[0]:
                            data_list = data[0]['data']
                        else:
                            data_list = data
                    elif isinstance(data, dict) and 'data' in data:
                        # {'totalcount': ..., 'data': [...]} 형태인 경우
                        data_list = data['data']
                    else:
                        data_list = [data] if isinstance(data, dict) else []
                    
                    # 각 논문 데이터 처리
                    for item in data_list:
                        # 필수 필드 확인
                        if 'doc_id' not in item or 'title' not in item:
                            continue
                        
                        # 초록 추출 (summary_entire 또는 summary_section 사용)
                        abstract = ""
                        if 'summary_entire' in item and item['summary_entire']:
                            # summary_entire의 첫 번째 원본 텍스트 사용
                            if len(item['summary_entire']) > 0:
                                abstract = item['summary_entire'][0].get('orginal_text', '')
                        
                        # 섹션 요약도 추가 (더 풍부한 검색을 위해)
                        section_texts = []
                        if 'summary_section' in item and item['summary_section']:
                            for section in item['summary_section']:
                                section_text = section.get('orginal_text', '')
                                if section_text:
                                    section_texts.append(section_text)
                        
                        # 논문 정보 구성
                        paper = {
                            'id': item['doc_id'],
                            'title': item['title'],
                            'abstract': abstract,
                            'content': ' '.join(section_texts) if section_texts else '',
                            'date': item.get('date', ''),
                            'author': item.get('author', ''),
                            'doc_type': item.get('doc_type', '논문'),
                            'ipc': item.get('ipc', ''),
                            'issued_by': item.get('issued_by', '')
                        }
                        
                        papers.append(paper)
                        total_loaded += 1
                        
            except Exception as e:
                print(f"  오류 발생 ({json_file}): {e}")
                import traceback
                traceback.print_exc()
                continue
    
    print(f"\n총 {total_loaded}개 논문 로드 완료!")
    
    if total_loaded > 0:
        print(f"데이터셋 분포:")
        
        # 분야별 통계
        ipc_counts = Counter([p['ipc'] for p in papers if p['ipc']])
        print(f"\n분야별 논문 수:")
        for ipc, count in ipc_counts.most_common(10):
            print(f"  - {ipc}: {count}개")
    
    return papers


In [42]:
# 딕셔너리 json 저장

def save_dic_json(dic_data, json_path):
    # JSON 파일로 저장
    # 출력 디렉토리 생성
    os.makedirs(os.path.dirname(json_path), exist_ok=True)
    # JSON으로 저장
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(dic_data, f, ensure_ascii=False, indent=2)
    print(f"저장 완료: ({len(dic_data)}) {json_path}")


def load_dic_json(json_path):
    """
    저장된 JSON 파일에서 논문 데이터 로드
    
    Args:
        json_path: JSON 파일 경로
    
    Returns:
        dic_data
    """
    if not os.path.exists(json_path):
        print(f"파일이 존재하지 않습니다: {json_path}")
        return []
    with open(json_path, 'r', encoding='utf-8') as f:
        dic_data = json.load(f)
    print(f"로드 완료: ({len(dic_data)}) {json_path}")
    return dic_data


In [34]:
# 3. 임베딩 모델 로드
def load_embedding_model(model_name='BM-K/KoSimCSE-roberta-multitask'):
    """
    임베딩 모델 로드
    
    추천 모델:
    - 'BM-K/KoSimCSE-roberta-multitask' (한국어 최적)
    - 'intfloat/multilingual-e5-large' (다국어 지원)
    - 'BAAI/bge-m3' (긴 문서 처리)
    """
    print(f"Loading model: {model_name}")
    model = SentenceTransformer(model_name)
    
    # GPU로 이동
    if torch.cuda.is_available():
        model = model.to('cuda')
    
    print(f"Model loaded successfully!")
    return model


In [35]:
# 4. 문서 임베딩 생성
def create_embeddings(papers, model, batch_size=32, use_title_and_abstract=True):
    """
    논문 임베딩 생성
    
    Args:
        papers: 논문 리스트
        model: 임베딩 모델
        batch_size: 배치 크기
        use_title_and_abstract: 제목+초록 사용 여부
    """
    texts = []
    for paper in papers:
        if use_title_and_abstract:
            # 제목과 초록 결합 (검색 성능 향상)
            text = f"{paper['title']} {paper['abstract']}"
        else:
            # 전체 내용 사용
            text = f"{paper['title']} {paper['abstract']} {paper.get('content', '')}"
        texts.append(text)
    
    print(f"Encoding {len(texts)} documents...")
    embeddings = model.encode(
        texts,
        batch_size=batch_size,
        show_progress_bar=True,
        convert_to_tensor=False,
        normalize_embeddings=True  # 코사인 유사도를 위한 정규화
    )
    
    return np.array(embeddings), texts


In [36]:
# 5. FAISS 인덱스 구축
def build_faiss_index(embeddings, use_gpu=True):
    """
    FAISS 인덱스 구축
    
    Args:
        embeddings: 임베딩 벡터
        use_gpu: GPU 사용 여부
    """
    dimension = embeddings.shape[1]
    
    # L2 정규화된 벡터에 내적 사용 = 코사인 유사도
    index = faiss.IndexFlatIP(dimension)
    
    # GPU 사용
    if use_gpu and torch.cuda.is_available():
        try:
            res = faiss.StandardGpuResources()
            index = faiss.index_cpu_to_gpu(res, 0, index)
        except Exception as e:
            print(f"GPU 사용 실패, CPU로 대체: {e}")
    
    # 임베딩 추가
    index.add(embeddings.astype('float32'))
    
    print(f"FAISS index built: {index.ntotal} vectors, dimension {dimension}")
    return index

# ============================================================================
# 6. 검색 함수
# ============================================================================

def search(query, model, index, papers, top_k=5):
    """
    논문 검색
    
    Args:
        query: 검색 쿼리
        model: 임베딩 모델
        index: FAISS 인덱스
        papers: 논문 리스트
        top_k: 반환할 결과 수
    """
    # 쿼리 임베딩
    query_embedding = model.encode(
        [query],
        convert_to_tensor=False,
        normalize_embeddings=True
    )
    
    # 검색
    scores, indices = index.search(query_embedding.astype('float32'), top_k)
    
    # 결과 포맷팅
    results = []
    for idx, score in zip(indices[0], scores[0]):
        paper = papers[idx]
        results.append({
            'paper_id': paper['id'],
            'title': paper['title'],
            'abstract': paper['abstract'][:200] + '...' if len(paper['abstract']) > 200 else paper['abstract'],
            'similarity_score': float(score),
            'author': paper.get('author', 'N/A'),
            'ipc': paper.get('ipc', 'N/A'),
            'date': paper.get('date', 'N/A')
        })
    
    return results


In [37]:
# 메인 실행 함수 (AI Hub 데이터)
def main_with_aihub_data():
    """AI Hub 실제 데이터로 실행"""
    
    print("="*80)
    print("한국어 논문 검색 시스템 - AI Hub 데이터")
    print("="*80)
    
    # 1. 실제 AI Hub 데이터 로드
    print("\n[1/6] Loading AI Hub papers...")
    papers = load_aihub_papers()
    
    if len(papers) == 0:
        print("로드된 논문이 없습니다. 경로를 확인해주세요.")
        return None, None, None
    
    print(f"Loaded {len(papers)} papers")
    
    # 샘플 논문 정보 출력
    print("\n샘플 논문 정보:")
    for i, paper in enumerate(papers[:3], 1):
        print(f"\n{i}. {paper['title']}")
        print(f"   ID: {paper['id']}")
        print(f"   저자: {paper['author']}")
        print(f"   분야: {paper['ipc']}")
        print(f"   초록: {paper['abstract'][:100]}...")
    
    # 2. 모델 로드
    print("\n[2/6] Loading embedding model...")
    model = load_embedding_model('BM-K/KoSimCSE-roberta-multitask')
    
    # 3. 임베딩 생성 (배치 크기를 줄여서 메모리 효율성 향상)
    print("\n[3/6] Creating embeddings...")
    embeddings, texts = create_embeddings(papers, model, batch_size=16)
    print(f"Created embeddings: shape {embeddings.shape}")
    
    # 4. FAISS 인덱스 구축
    print("\n[4/6] Building FAISS index...")
    index = build_faiss_index(embeddings, use_gpu=torch.cuda.is_available())
    
    # 5. 검색 테스트
    print("\n[5/6] Testing search...")
    test_queries = [
        "딥러닝 자연어 처리",
        "컴퓨터 비전 객체 인식",
        "강화학습 인공지능",
        "데이터 분석 머신러닝"
    ]
    
    for query in test_queries:
        print(f"\n검색어: '{query}'")
        print("-" * 80)
        results = search(query, model, index, papers, top_k=5)
        
        for i, result in enumerate(results, 1):
            print(f"{i}. [{result['similarity_score']:.4f}] {result['title']}")
            print(f"   저자: {result['author']}")
            print(f"   분야: {result['ipc']}")
            print(f"   초록: {result['abstract'][:150]}")
            print()
    
    # # 6. 인터랙티브 검색
    # print("\n[6/6] Interactive search mode")
    # print("검색어를 입력하세요 (종료: 'quit')")
    
    # while True:
    #     query = input("\n검색어: ").strip()
    #     if query.lower() == 'quit':
    #         break
        
    #     if not query:
    #         continue
        
    #     results = search(query, model, index, papers, top_k=10)
    #     print(f"\n검색 결과 ({len(results)}개):")
    #     print("="*80)
        
    #     for i, result in enumerate(results, 1):
    #         print(f"{i}. [{result['similarity_score']:.4f}] {result['title']}")
    #         print(f"   저자: {result['author']}")
    #         print(f"   분야: {result['ipc']}")
    #         print(f"   발행: {result.get('issued_by', 'N/A')} ({result['date']})")
    #         print(f"   초록: {result['abstract']}")
    #         print()
    
    print("\n프로그램을 종료합니다.")
    
    return papers, model, index

# ============================================================================
# 실행 예시
# ============================================================================

#papers, model, index = main_with_aihub_data()


In [38]:
raise ValueError('여기까지')

ValueError: 여기까지

In [39]:
# 1. 실제 AI Hub 데이터 로드
papers = load_aihub_papers()

데이터 로드 중...
처리 중: D:\dataset\018.논문자료_요약_데이터\01.데이터\1._Training\1._라벨링데이터_231101_add\training_논문.zip.unzip
  찾은 JSON 파일: 4개


로딩 중:   0%|          | 0/4 [00:00<?, ?it/s]

처리 중: D:\dataset\018.논문자료_요약_데이터\01.데이터\2._Validation\1._라벨링데이터_231101_add\validation_논문.zip.unzip
  찾은 JSON 파일: 1개


로딩 중:   0%|          | 0/1 [00:00<?, ?it/s]


총 162087개 논문 로드 완료!
데이터셋 분포:

분야별 논문 수:
  - 사회과학: 77736개
  - 인문학: 32433개
  - 공학: 16531개
  - 복합학: 12150개
  - 예술체육학: 7771개
  - 의약학: 7430개
  - 농수해양학: 4198개
  - 자연과학: 3838개


In [40]:
save_dic_json(papers, r"d:\temp\논문.json")

저장 완료: (162087) d:\temp\논문.json


In [43]:
papers = load_dic_json(r"d:\temp\논문.json")
print('papers = ', len(papers))

로드 완료: (162087) d:\temp\논문.json
papers =  162087


In [44]:
import gdown
def download_gdrive_file(url, output_path, ignore=True):
    """Google Drive 파일 다운로드 함수

    Args:
        url (str): Google Drive 공유 링크
        output_path (str): 다운로드할 파일 경로
        ignore (bool, optional): True면 기존 파일 삭제 후 다운로드, False면 파일 있으면 건너뜀. Defaults to True.

    Raises:
        ValueError: Google Drive 파일 ID를 찾을 수 없습니다.
    """
    # 공유 링크에서 파일 ID 추출
    if os.path.exists(output_path):
        if ignore:
            os.remove(output_path)
        else:
            return

    file_id_match = re.search(r'/d/([a-zA-Z0-9_-]+)', url)
    if not file_id_match:
        raise ValueError("Google Drive 파일 ID를 찾을 수 없습니다.")
    file_id = file_id_match.group(1)
    gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)

def download_http(url, output_path, ignore=True):
    """
    HTTP 파일 다운로드 함수 (진행률 표시)
    url: 다운로드할 파일 URL
    output_path: 저장할 파일 경로
    ignore: True면 기존 파일 삭제 후 다운로드, False면 파일 있으면 건너뜀
    """
    if os.path.exists(output_path):
        if ignore:
            os.remove(output_path)
        else:
            print(f"이미 파일이 존재합니다: {output_path}")
            return output_path

    response = requests.get(url, stream=True)
    total = int(response.headers.get('content-length', 0))
    with open(output_path, 'wb') as file, tqdm(
        desc=f"Downloading {os.path.basename(output_path)}",
        total=total,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
        ascii=True
    ) as bar:
        for data in response.iter_content(chunk_size=1024):
            size = file.write(data)
            bar.update(size)
    print(f"다운로드 완료: {output_path}")
    return output_path

In [51]:
if  IS_COLAB:
    local_papers = os.path.join( '/content/', 'papers')
else:
    local_papers = os.path.join( Path.cwd().drive + '\\\\', 'temp', 'papers')
os.makedirs(local_papers, exist_ok=True)  # 폴더 생성 코드 추가
zipfile = os.path.join(local_papers, r'dataset_papers.zip')

# https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzk0MDg2NDIwODQ4MHxGfDA&svcType=MYBOX-WEB&time=1760347412747
download_http(r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzk0MDg2NDIwODQ4MHxGfDA&svcType=MYBOX-WEB&time=1760347412747',
              zipfile)
unzip_paths = unzip([zipfile])


다운로드 완료: d:\\temp\papers\dataset_papers.zip
이미 압축 해제됨: d:\\temp\papers\dataset_papers.zip.unzip


In [52]:
papers_json_path = os.path.join(unzip_paths[0], 'papers.json')
print(papers_json_path, os.path.exists(papers_json_path))

d:\\temp\papers\dataset_papers.zip.unzip\papers.json True


In [58]:
papers = load_dic_json(papers_json_path)


로드 완료: (162087) d:\\temp\papers\dataset_papers.zip.unzip\papers.json


In [59]:



print(f"Loaded {len(papers)} papers")

# 샘플 논문 정보 출력
print("\n샘플 논문 정보:")
for i, paper in enumerate(papers[:3], 1):
    print(f"\n{i}. {paper['title']}")
    print(f"   ID: {paper['id']}")
    print(f"   저자: {paper['author']}")
    print(f"   분야: {paper['ipc']}")
    print(f"   초록: {paper['abstract'][:100]}...")

# 2. 모델 로드
print("\n[2/6] Loading embedding model...")
model = load_embedding_model('BM-K/KoSimCSE-roberta-multitask')

# 3. 임베딩 생성 (배치 크기를 줄여서 메모리 효율성 향상)
print("\n[3/6] Creating embeddings...")
embeddings, texts = create_embeddings(papers, model, batch_size=16)
print(f"Created embeddings: shape {embeddings.shape}")

# 4. FAISS 인덱스 구축
print("\n[4/6] Building FAISS index...")
index = build_faiss_index(embeddings, use_gpu=torch.cuda.is_available())

# 5. 검색 테스트
print("\n[5/6] Testing search...")
test_queries = [
    "딥러닝 자연어 처리",
    "컴퓨터 비전 객체 인식",
    "강화학습 인공지능",
    "데이터 분석 머신러닝"
]

for query in test_queries:
    print(f"\n검색어: '{query}'")
    print("-" * 80)
    results = search(query, model, index, papers, top_k=5)
    
    for i, result in enumerate(results, 1):
        print(f"{i}. [{result['similarity_score']:.4f}] {result['title']}")
        print(f"   저자: {result['author']}")
        print(f"   분야: {result['ipc']}")
        print(f"   초록: {result['abstract'][:150]}")
        print()


Loaded 162087 papers

샘플 논문 정보:

1. 중세시대의 對句 학습과 문학 교육
   ID: A201007089365
   저자: 김성룡
   분야: 인문학
   초록: 짐작할 수 있는 것처럼, 對句의 영역은 음운, 단어, 어구, 시편의 차원에서 다양하게 이뤄지고 있으며, 그 원리는 짝짓기이다. 하나의 텍스트는 음운으로부터 텍스트 자체까지에 이르는...

2. 일제하 토지회수운동의 전개과정 - 전남 무안군 하의도의 사례 -
   ID: A201007089374
   저자: 이규수
   분야: 인문학
   초록: 다음과 같다. 하의도에 상륙한 신한공사 사원과 경관 등은 소작료를 강제 징수하기 위해 2개조로 나뉘어, 제1대는 오림리, 제2대는 대리로 향했다. 오림리에 도착한 제1대는 곧바로 ...

3. 코스닥시장 가격제한폭 변화가 주가지수 변동성에 미치는 영향
   ID: A201007089661
   저자: 유한수
   분야: 사회과학
   초록: 코스닥시장에서 가격제한폭 변화가 이루어진 1998. 5. 25의 경우를 대상으로 분석하였다. 가격제한폭 변경일을 기준으로 가격제한폭 변경전 60일간을 기간 A, 변경후 60일간을 ...

[2/6] Loading embedding model...
Loading model: BM-K/KoSimCSE-roberta-multitask


2025-10-13 18:57:27 [W] No sentence-transformers model found with name BM-K/KoSimCSE-roberta-multitask. Creating a new one with mean pooling.


Model loaded successfully!

[3/6] Creating embeddings...
Encoding 162087 documents...


Batches:   0%|          | 0/10131 [00:00<?, ?it/s]

KeyboardInterrupt: 